<a href="https://colab.research.google.com/github/happyBeagle/MNIST_with_kaggle/blob/main/MNIST_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/happyBeagle/MNIST_with_kaggle.git
!ls
!cd MNIST_with_kaggle/

Cloning into 'MNIST_with_kaggle'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
MNIST_with_kaggle  sample_data


In [2]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"happybeagle","key":"710b358482820650f28cf9dda7e0c8ff"}'}

In [3]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 Jan  9 10:07 kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c digit-recognizer

 55% 5.00M/9.16M [00:00<00:00, 18.5MB/s]
100% 9.16M/9.16M [00:00<00:00, 26.4MB/s]
 82% 5.00M/6.09M [00:00<00:00, 26.9MB/s]
100% 6.09M/6.09M [00:00<00:00, 29.8MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 121MB/s]


In [6]:
!ls
!unzip test.csv.zip
!unzip train.csv.zip

kaggle.json	   sample_data		  test.csv.zip
MNIST_with_kaggle  sample_submission.csv  train.csv.zip
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [7]:
!ls
!mkdir ./data
!mv test.csv ./data/
!mv train.csv ./data/
!mv sample_submission.csv ./data/

kaggle.json	   sample_data		  test.csv	train.csv
MNIST_with_kaggle  sample_submission.csv  test.csv.zip	train.csv.zip


In [8]:
!ls ./data/

sample_submission.csv  test.csv  train.csv


In [9]:
import numpy as np
import pandas as pd


import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from torch.utils.data import DataLoader


import os
print(os.listdir("./data"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

['train.csv', 'sample_submission.csv', 'test.csv']


In [30]:

train_data = pd.read_csv(r"./data/train.csv",dtype = np.float32)


#split feature and label
label = train_data.label.values
features = train_data.loc[:, train_data.columns != "label"].values/255.0

train_features, test_features, train_label, test_label = train_test_split(features, label,
                                                                          test_size = 0.2,
                                                                          random_state = 42)

trainFeatures = torch.from_numpy(train_features)
trainLabel = torch.from_numpy(train_label).type(torch.LongTensor)

testFeatures = torch.from_numpy(test_features)
testLabel = torch.from_numpy(test_label).type(torch.LongTensor)

#batch size, epoch and Iteration
batch_size = 100
n_iterators = 100000
num_epochs = n_iterators / (len(train_features) / batch_size)
num_epochs = int(num_epochs) 

# Pytorch datasets
train = torch.utils.data.TensorDataset(trainFeatures, trainLabel)

test = torch.utils.data.TensorDataset(testFeatures, testLabel)

# data loader
train_loader = DataLoader(train, batch_size=batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)


In [31]:
#Logistic regression

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

In [32]:
input_dim = 28 * 28
output_dim = 10


model = LogisticRegressionModel(input_dim, output_dim)

error = nn.CrossEntropyLoss()

learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
count = 0
for e in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        Train = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(Train)

        loss = error(outputs, labels)

        loss.backward()

        optimizer.step()
        count += 1

        if count % 50 == 0:
            correct, total = 0, 0

            for images, labels in test_loader:
                test = Variable(images.view(-1, 28 * 28))
                outputs = model(test)
                predicted = torch.max(outputs.data, 1)[1]
                total += len(labels)
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / float(total)

        if count % 500 == 0:
            print("Iteration: {}  Loss : {}   Accuracy : {}%".format(count, loss.data, accuracy))

Iteration: 500  Loss : 1.8341602087020874   Accuracy : 67.1547622680664%
Iteration: 1000  Loss : 1.6048409938812256   Accuracy : 75.03571319580078%
Iteration: 1500  Loss : 1.3026679754257202   Accuracy : 77.94047546386719%
Iteration: 2000  Loss : 1.2123323678970337   Accuracy : 79.52381134033203%
Iteration: 2500  Loss : 1.036089301109314   Accuracy : 80.6547622680664%
Iteration: 3000  Loss : 0.9283428192138672   Accuracy : 81.58333587646484%
Iteration: 3500  Loss : 0.9065825939178467   Accuracy : 82.35713958740234%
Iteration: 4000  Loss : 0.7546180486679077   Accuracy : 82.83333587646484%
Iteration: 4500  Loss : 0.9631974101066589   Accuracy : 83.36904907226562%
Iteration: 5000  Loss : 0.8028979301452637   Accuracy : 83.76190185546875%
Iteration: 5500  Loss : 0.7532106637954712   Accuracy : 84.14286041259766%
Iteration: 6000  Loss : 0.8718149065971375   Accuracy : 84.38095092773438%
Iteration: 6500  Loss : 0.671949028968811   Accuracy : 84.78571319580078%
Iteration: 7000  Loss : 0.7082

In [34]:
submit_data = pd.read_csv('./data/test.csv',dtype = np.float32)
submit_data = submit_data.values/255.0
submit_loader = torch.utils.data.DataLoader(dataset=submit_data, batch_size=batch_size, shuffle=False)

In [37]:
def prediction(data_loader):
    model.eval()
    test_pred = torch.LongTensor()

    for i, data in enumerate(data_loader):
        data = Variable(data.view(-1, 28*28))
        
        output = model(data)
        pred = torch.max(output.data, 1)[1]

        test_pred = torch.cat((test_pred, pred), dim=0)

    return test_pred

In [41]:
test_pred = prediction(submit_loader)

output_df = pd.DataFrame(np.c_[np.arange(1, len(submit_data)+1)[:,None], test_pred.numpy()], 
                      columns=['ImageId', 'Label'])

output_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [42]:
output_df.to_csv('submission.csv', index=False)